In [1]:
import os
import sys

root = os.path.abspath(os.path.join('..', '..'))
test_dir = os.path.join(root, 'backend', 'docking', 'tests')

In [ ]:
# We can use rdkit to generate small molecule conformers necessary
# for Rosetta ligand docking:

import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign

# Take example output from this directory:
example_hits = '2gs6_picks.sdf'
max_confs = 10

def heavy_atom_map(prb, ref):
    """Build (prb_idx, ref_idx) mapping for non-H (heavy) atoms.
    Assumes AddHs preserves the ordering of the original heavy atoms in the produced molecule.
    """
    prb_heavy = [a.GetIdx() for a in prb.GetAtoms() if a.GetAtomicNum() != 1]
    ref_heavy = [a.GetIdx() for a in ref.GetAtoms() if a.GetAtomicNum() != 1]
    if len(prb_heavy) != len(ref_heavy):
        raise ValueError("Mismatch in number of heavy atoms between probe and reference")
    return list(zip(prb_heavy, ref_heavy))

def align_conformers_to_ref(prb, ref, conf_ids):
    """Align each conformer of mol_Hs to the reference molecule (ref) using heavy-atom mapping."""
    mapping = heavy_atom_map(prb, ref)
    ref_conf_id = ref.GetConformer().GetId()
    for cid in conf_ids:
        rdMolAlign.AlignMol(prb, ref, prbCid=cid, refCid=ref_conf_id, atomMap=mapping)

def hits_to_conformers(hits_sdf, max_confs_per_hit=10):
    ''' Output conformers for each hit ligand '''
    
    supplier = Chem.SDMolSupplier(hits_sdf)
    outdir = os.path.dirname(hits_sdf)
    hits_tag = os.path.basename(os.path.splitext(hits_sdf)[0])
    pathlist = []

    for i, mol in enumerate(supplier):
        if mol is None:
            continue

        # Track hit structure data:
        hit_name = str(i+1).zfill(3)
        hit_ref = Chem.Mol(mol)

        # Add hydrogens to the hit structure:
        mol = Chem.AddHs(mol, addCoords=True)

        # Generate conformers for each hit:
        conf_ids = AllChem.EmbedMultipleConfs(
            mol,
            numConfs=max_confs_per_hit,
            useExpTorsionAnglePrefs=True,
            useBasicKnowledge=True
        )

        # Align each conf to hit coords scaffold:
        align_conformers_to_ref(mol, hit_ref, conf_ids=conf_ids)

        out_file = os.path.join(outdir, f'{hits_tag}_{hit_name}_conformers.sdf')
        with Chem.SDWriter(out_file) as writer:
            for conf_id in conf_ids:
                writer.write(mol, confId=conf_id)
        pathlist.append(out_file)

    return pathlist

pathlist = hits_to_conformers(example_hits, max_confs)

In [ ]:
# Now we use Rosetta's molfile_to_params.py script to generate
# a parameters file for each ligand in addition to a pdb file
# for the hit ligand and another for each of its conformers.

# Note, this will only work if running with an installed Rosetta version.

import subprocess
import glob

conf_files = sorted(glob.glob(os.path.join(test_dir, '2gs6_hit_*_conformers.sdf')))

script_path = os.path.join(test_dir, '..', 'molfile_to_params.py')
script_path = os.path.abspath(script_path)

for i, file in enumerate(conf_files):
    lig_pdb_tag = f'NM{i+1}'
    prefix = os.path.basename(os.path.splitext(file)[0])
    cmd = [
        sys.executable, script_path,
        '-n', lig_pdb_tag,
        '-p', prefix,
        '--chain=X',
        '--conformers-in-one-file', file
    ]
    subprocess.run(cmd, check=True, cwd=test_dir)

In [11]:
# Concatenate the prepared ligand pdb and its receptor:

for i in range(1, 13):
    rec_pdb_file = '2gs6_receptor.pdb'
    lig_pdb_file = os.path.join(test_dir, f'2gs6_hit_{i}.pdb')
    complex_pdb_file = os.path.join(test_dir, f'2gs6_hit_{i}_complex.pdb')

    with open(rec_pdb_file, 'r') as f_rec, open(lig_pdb_file, 'r') as f_lig, open(complex_pdb_file, 'w') as f_out:
        # Write receptor atoms first:
        for line in f_rec:
            if line.startswith('ATOM'):
                f_out.write(line)
        f_out.write('TER\n')

        # Write ligand atoms second:
        for line in f_lig:
            if line.startswith('HETATM'):
                f_out.write(line)
        f_out.write('END\n')
